In [1]:
import scraper as s
import pandas as pd
import numpy as np
import datetime
m = s.stockMongo()

In [2]:
op = m.get_options("AAPL")

In [3]:
op.head()

,ask,bid,contract,in-money,iv,last,last_trade,strike,strike-date,type,volume
date,,,,,,,,,,,
2020-08-02 23:39:37.026,332.30,327.95,AAPL200918C00095000,in-the-money,148.44%,284.24,2020-07-30,95.0,2020-09-18,Call,2.0
2020-08-02 23:39:37.026,279.25,277.65,AAPL200918C00100000,in-the-money,0.00%,254.30,2020-06-29,100.0,2020-09-18,Call,1.0
2020-08-02 23:39:37.026,322.50,318.00,AAPL200918C00105000,in-the-money,152.54%,264.25,2020-07-23,105.0,2020-09-18,Call,1.0
2020-08-02 23:39:37.026,317.30,312.90,AAPL200918C00110000,in-the-money,129.69%,266.25,2020-07-27,110.0,2020-09-18,Call,1.0
2020-08-02 23:39:37.026,258.90,254.10,AAPL200918C00115000,in-the-money,0.00%,255.30,2020-06-23,115.0,2020-09-18,Call,2.0


In [28]:
apple = op.where(op['last_trade']==op.index.date).dropna()
apple = apple.drop(['volume', 'iv', 'in-money', 'bid', 'ask','last_trade', 'contract'], axis=1)
apple = apple.reset_index()
apple = apple.set_index(['strike-date', 'type'])
apple['date'] = pd.to_datetime(apple['date']).dt.strftime('%m-%d-%Y')
apple['date'] = pd.to_datetime(apple['date'])
strike_pivot = apple.pivot_table(columns="strike-date", values="date", aggfunc=np.count_nonzero)
strike_cols = strike_pivot.columns
strike_cols

DatetimeIndex(['2020-08-07', '2020-08-14', '2020-08-21', '2020-08-28',
               '2020-09-04', '2020-09-11', '2020-09-18', '2020-09-25',
               '2020-10-02', '2020-10-09', '2020-10-16', '2020-11-20',
               '2020-12-18', '2021-01-15', '2021-06-18', '2021-09-17',
               '2022-01-21', '2022-06-17', '2022-09-16'],
              dtype='datetime64[ns]', name='strike-date', freq=None)

In [31]:
new_apple = []
apple2 = apple

apple2 = apple2.reset_index()
apple2 = apple2.set_index(['type','strike-date'])
apple_pivot_call = apple2.loc[('Call', strike_cols[8])]
strike_pivot = apple_pivot_call.pivot_table(columns="strike", values="date", aggfunc=np.count_nonzero)
strike_value_cols = strike_pivot.columns
apple_pivot_put = apple2.loc[('Put', strike_cols[8])]
strike_pivot = apple_pivot_put.pivot_table(columns="strike", values="date", aggfunc=np.count_nonzero)
strike_value_puts = strike_pivot.columns
    
apple2 = apple2.reset_index()
apple2 = apple2.set_index(['type', 'strike','strike-date'])

i = 0
j = 0

for i in range(len(strike_value_cols)-1):
    print(i)
    for j in range(len(strike_value_puts)-1):
        aCLo = apple2.loc[('Call', strike_value_cols[i], strike_cols[8])]
        aCLo = aCLo.reset_index()
        #aCLo = aCLo.drop(['type', 'strike-date'], axis=1)
        aCLo.rename(columns = {'last':'priceCLo', 'strike': 'strikeCLo'}, inplace = True)
        aCHi = apple2.loc[('Call', strike_value_cols[i+1], strike_cols[8])]
        aCHi = aCHi.reset_index()
        #aCHi = aCHi.drop(['type', 'strike-date'], axis=1)
        aCHi.rename(columns = {'last':'priceCHi', 'strike': 'strikeCHi'}, inplace = True)
        aPLo = apple2.loc[('Put', strike_value_puts[j], strike_cols[8])]
        aPLo = aPLo.reset_index()
        #aPLo = aPLo.drop(['type', 'strike-date'], axis=1)
        aPLo.rename(columns = {'last':'pricePLo', 'strike': 'strikePLo'}, inplace = True)
        aPHi = apple2.loc[('Put', strike_value_puts[j+1], strike_cols[8])]
        aPHi = aPHi.reset_index()
        #aPHi = aPHi.drop(['type', 'strike-date'], axis=1)
        aPHi.rename(columns = {'last':'pricePHi', 'strike': 'strikePHi'}, inplace = True)

        a = pd.merge(aCLo, aCHi, on=['date', 'strike-date'])
        a = pd.merge(a, aPLo, on=['date', 'strike-date'])
        a = pd.merge(a, aPHi, on=['date', 'strike-date'])
        a = pd.merge(a, c, on='date')
        a = a.drop(['type_x', 'type_y'], axis=1)

        new_apple.append(a)

print(new_apple)


c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  if __name__ == '__main__':
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:22: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:26: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:30: PerformanceWarning: indexing past lexsort depth may impact performance.
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:34: PerformanceWarning: indexing past lexsort depth may impact performance

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
[Empty DataFrame
Columns: [strikeCLo, priceCLo, strikeCHi, priceCHi, strikePLo, pricePLo, strikePHi, strike-date, pricePHi, c, date]
Index: [], Empty DataFrame
Columns: [strikeCLo, priceCLo, strikeCHi, priceCHi, strikePLo, pricePLo, strikePHi, strike-date, pricePHi, c, date]
Index: [], Empty DataFrame
Columns: [strikeCLo, priceCLo, strikeCHi, priceCHi, strikePLo, pricePLo, strikePHi, strike-date, pricePHi, c, date]
Index: [], Empty DataFrame
Columns: [strikeCLo, priceCLo, strikeCHi, priceCHi, strikePLo, pricePLo, strikePHi, strike-date, pricePHi, c, date]
Index: [], Empty DataFrame
Colum

In [32]:
new_apple = pd.concat(new_apple)

In [33]:
new_apple

,strikeCLo,priceCLo,strikeCHi,priceCHi,strikePLo,pricePLo,strikePHi,strike-date,pricePHi,c,date
0,90.0,25.50,91.25,25.50,63.75,0.08,65.00,2020-10-02,0.08,113.489998,2020-09-10
0,90.0,25.50,91.25,25.50,72.50,0.10,73.75,2020-10-02,0.13,113.489998,2020-09-10
0,90.0,25.50,91.25,25.50,73.75,0.13,75.00,2020-10-02,0.14,113.489998,2020-09-10
0,90.0,25.50,91.25,25.50,75.00,0.14,77.50,2020-10-02,0.16,113.489998,2020-09-10
0,90.0,25.50,91.25,25.50,77.50,0.16,78.75,2020-10-02,0.20,113.489998,2020-09-10
...,...,...,...,...,...,...,...,...,...,...,...
1,670.0,3.70,680.00,3.60,510.00,35.30,515.00,2020-10-02,35.75,126.522499,2020-08-26
0,670.0,2.98,680.00,2.71,515.00,40.95,520.00,2020-10-02,44.25,124.824997,2020-08-25
0,670.0,2.98,680.00,2.71,520.00,44.25,560.00,2020-10-02,74.77,124.824997,2020-08-25
0,670.0,2.98,680.00,2.71,560.00,74.77,610.00,2020-10-02,119.74,124.824997,2020-08-25


In [30]:
new_apple = []
apple2 = apple

apple2 = apple2.reset_index()
apple2 = apple2.set_index(['type','strike-date'])
apple_pivot_call = apple2.loc[('Call', strike_cols[8])]
strike_pivot = apple_pivot_call.pivot_table(columns="strike", values="date", aggfunc=np.count_nonzero)
strike_value_cols = strike_pivot.columns
apple_pivot_put = apple2.loc[('Put', strike_cols[8])]
strike_pivot = apple_pivot_put.pivot_table(columns="strike", values="date", aggfunc=np.count_nonzero)
strike_value_puts = strike_pivot.columns

print(len(strike_value_cols))
print(len(strike_value_puts))

130
110


c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:6: PerformanceWarning: indexing past lexsort depth may impact performance.
  
c:\userdata\z003rvhr\python\yahoo_graber\grab\lib\site-packages\ipykernel_launcher.py:9: PerformanceWarning: indexing past lexsort depth may impact performance.
  if __name__ == '__main__':
